In [1]:
import ccxt
import pprint

# Bybit Testnet에 연결
exchange = ccxt.bybit({
    'apiKey': '36Eh4304sQhAm28mjI',  # 여기에 API 키 입력
    'secret': '7aQOPDw0FwnjwUMGE5PNWyXxgUvlL5tfdbus',  # 여기에 API 시크릿 입력
    'enableRateLimit': True,
})

# Testnet 설정
exchange.set_sandbox_mode(True)

# OHLCV 데이터 가져오기
symbol = 'BTC/USDT'  # 원하는 거래 쌍을 여기에 입력
timeframe = '1h'  # 사용할 시간 프레임 (예: '1m', '5m', '1h', '1d')
limit = 2000  # 가져올 데이터 포인트 수

# 데이터 요청
ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)

# 결과 출력
pprint.pprint(ohlcv)


[[1698703200000, 30000.04, 30103.99, 30000.03, 30000.06, 122.629655],
 [1698706800000, 30000.06, 30113.64, 30000.03, 30000.07, 122.958798],
 [1698710400000, 30000.07, 30103.99, 30000.03, 30000.06, 122.024758],
 [1698714000000, 30000.06, 30165.0, 30000.03, 30000.05, 123.555709],
 [1698717600000, 30000.05, 30103.99, 30000.03, 30000.05, 118.084331],
 [1698721200000, 30000.05, 30113.64, 30000.02, 30000.04, 123.029455],
 [1698724800000, 30000.04, 30118.0, 30000.03, 30000.06, 63.742439],
 [1698728400000, 30000.06, 30162.08, 30000.04, 30103.99, 39.415774],
 [1698732000000, 30103.99, 30165.0, 30000.04, 30000.08, 37.086038],
 [1698735600000, 30000.08, 30184.82, 30000.04, 30077.86, 40.536924],
 [1698739200000, 30077.86, 30172.95, 30000.04, 30000.07, 48.645303],
 [1698742800000, 30000.07, 31993.12, 30000.01, 31944.9, 830.594736],
 [1698746400000, 31944.9, 31945.06, 31639.25, 31639.25, 39.767498],
 [1698750000000, 31639.25, 31740.0, 31530.0, 31561.59, 34.165047],
 [1698753600000, 31561.59, 31945.1

In [ ]:
# 계좌 정보 조회
balance = exchange.fetch_balance()

# 결과 출력
pprint.pprint(balance)


In [ ]:
since = None
# 거래 내역 요청
trades = exchange.fetch_my_trades('BTC/USDT', since, limit)

# 결과 출력
pprint.pprint(trades)

In [ ]:
# 모든 시장 정보 로드
markets = exchange.load_markets()

# USDT 마켓의 거래 가능 종목 리스트 추출
usdt_pairs = {symbol: market for symbol, market in markets.items() if ':USDT' in symbol}

# 결과 출력
pprint.pprint(usdt_pairs)

In [6]:
len(usdt_pairs)

272

In [ ]:
# 시장 가격 가져오기
def get_price(symbol):
    ticker = exchange.fetch_ticker(symbol)
    return ticker['last']  # 현재 시장 가격

# 매수 주문 (USDT 기준)
def place_buy_order_usdt(symbol, amount_in_usdt):
    price = get_price(symbol)  # 현재 시장 가격 가져오기
    amount_in_coin = amount_in_usdt / price  # 코인의 양 계산
    order = exchange.create_order(symbol, 'market', 'buy', amount_in_coin, price)
    pprint.pprint(order)


# 매도 주문 (USDT 기준)
def place_sell_order_usdt(symbol, amount_in_usdt):
    price = get_price(symbol)
    amount_in_coin = amount_in_usdt / price  # 코인의 양 계산
    order = exchange.create_order(symbol, 'market', 'sell', amount_in_coin, price)
    pprint.pprint(order)

# 예제 사용
# place_buy_order_usdt('BTC/USDT', 100)  # BTC/USDT에 대한 50 USDT의 매수 주문
# place_sell_order_usdt('BTC/USDT', 20) # BTC/USDT에 대한 50 USDT의 매도 주문


In [ ]:
# 자산 조회
balance = exchange.fetch_balance()
print(balance)

# 거래 내역 조회
trades = exchange.fetch_my_trades(symbol='BTC/USDT')
for trade in trades:
    print(trade)


In [8]:
import pandas as pd

# DataFrame으로 변환
df = pd.DataFrame(balance['total'].items(), columns=['Currency', 'Total'])
df['Free'] = df['Currency'].map(balance['free'])
df['Used'] = df['Currency'].map(balance['used'])

# 필요한 정보 출력
print(df)

  Currency        Total         Free       Used
0      BTC     0.002275     0.002275   0.000000
1      ETH     0.000004     0.000004   0.000000
2     USDT  5939.451715  5839.807283  99.644432


In [9]:
# 자산 조회
balance = exchange.fetch_balance()
print(balance)

# 모든 코인에 대한 시장 가격을 USDT로 조회
prices = exchange.fetch_tickers()

# DataFrame 생성
df = pd.DataFrame(columns=['Currency', 'Total', 'Value in USDT'])

rows = []

for currency, total in balance['total'].items():
    if total > 0:  # 보유량이 있는 코인만 처리
        symbol = currency + '/USDT'
        if symbol in prices:  # USDT로 환산 가능한 코인만 처리
            usdt_value = total * prices[symbol]['last']  # 현재 가격으로 환산
            rows.append({
                'Currency': currency,
                'Total': total,
                'Value in USDT': usdt_value
            })

# DataFrame 생성
df = pd.DataFrame(rows, columns=['Currency', 'Total', 'Value in USDT'])

# 필요한 정보 출력
print(df)

{'info': {'retCode': '0', 'retMsg': 'OK', 'result': {'list': [{'totalEquity': '6034.39095382', 'accountIMRate': '0.0169', 'totalMarginBalance': '5938.75868632', 'totalInitialMargin': '100.58328551', 'accountType': 'UNIFIED', 'totalAvailableBalance': '5838.1754008', 'accountMMRate': '0.0013', 'totalPerpUPL': '0.80329757', 'totalWalletBalance': '5937.95538875', 'accountLTV': '0', 'totalMaintenanceMargin': '8.10325063', 'coin': [{'availableToBorrow': '', 'bonus': '0', 'accruedInterest': '0', 'availableToWithdraw': '0.00227543', 'totalOrderIM': '0', 'equity': '0.00227543', 'totalPositionMM': '0', 'usdValue': '95.62277593', 'unrealisedPnl': '0', 'collateralSwitch': False, 'spotHedgingQty': '0', 'borrowAmount': '0.000000000000000000', 'totalPositionIM': '0', 'walletBalance': '0.00227543', 'cumRealisedPnl': '-0.00002456', 'locked': '0', 'marginCollateral': True, 'coin': 'BTC'}, {'availableToBorrow': '', 'bonus': '0', 'accruedInterest': '0', 'availableToWithdraw': '0.00000425', 'totalOrderIM':

In [10]:
# Bybit 인스턴스 생성
exchange = ccxt.bybit({
    'apiKey': '36Eh4304sQhAm28mjI',  # 여기에 API 키 입력
    'secret': '7aQOPDw0FwnjwUMGE5PNWyXxgUvlL5tfdbus',  # 여기에 API 시크릿 입력
    'enableRateLimit': True,
    'createMarketBuyOrderRequiresPrice': False,
    'options': {
        'defaultType': 'swap',
    }
})

# Testnet 설정
exchange.set_sandbox_mode(True)

# 현재 잔액 조회
balance = exchange.fetch_balance()
current_balance = balance['total']['USDT']  # 예시로 USDT 잔액을 사용

# 초기 잔액 (예를 들어, 거래 시작 시점의 잔액)
initial_balance = 6000  # 이 값은 사용자가 직접 설정

asset_change_rate = ((current_balance - initial_balance) / initial_balance) * 100
print("자산 변화율:", asset_change_rate, "%")



자산 변화율: -1.0091380863333295 %
